<a href="https://colab.research.google.com/github/Oliivkaaaa/BigDate_/blob/main/Kafka_%D0%A3%D0%BD%D0%B8%D0%BA%D0%B0%D0%BB%D1%8C%D0%BD%D0%B0%D1%8F_%D0%BE%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0_%D1%81%D0%BE%D0%B1%D1%8B%D1%82%D0%B8%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from multiprocessing import Process, Queue
import time, json, random, os

TOPIC = "sensor_temperature_alerts"

def producer_task(q: Queue, n=10, delay=1.0):
    sensors = ["sensor-A", "sensor-B", "sensor-C"]
    for i in range(n):
        event = {
            "type": "TemperatureEvent",
            "topic": TOPIC,
            "sensor_id": random.choice(sensors),
            "temperature": round(random.uniform(15, 40), 2),
            "timestamp": time.time()
        }
        q.put(json.dumps(event))
        print(f"Producer -> sent: {event}")
        time.sleep(delay)
    q.put("__EOF__")

def consumer_task(q: Queue, out_log="consumer_sim.log"):
    with open(out_log, "w", encoding="utf-8") as f:
        print("Consumer started (simulator). Listening...")
        while True:
            raw = q.get()
            if raw == "__EOF__":
                print("Consumer received EOF, exiting.")
                break
            event = json.loads(raw)
            if event.get("topic") != TOPIC:
                continue
            if event.get("type") == "TemperatureEvent":
                temp = event.get("temperature")
                sensor = event.get("sensor_id")
                ts = event.get("timestamp")
                if temp > 30:
                    line = f"ALERT! {sensor} temperature = {temp}°C (ts={ts})"
                else:
                    line = f"{sensor} normal temperature = {temp}°C (ts={ts})"
                print(line)
                f.write(line + "\n")
                f.flush()

if __name__ == "__main__":
    q = Queue()
    p = Process(target=producer_task, args=(q,10,0.5))
    c = Process(target=consumer_task, args=(q,"consumer_sim.log"))
    c.start()
    p.start()
    p.join()
    c.join()
    print("Simulation finished. Logs written to consumer_sim.log")


Consumer started (simulator). Listening...
Producer -> sent: {'type': 'TemperatureEvent', 'topic': 'sensor_temperature_alerts', 'sensor_id': 'sensor-A', 'temperature': 24.35, 'timestamp': 1761201604.5129309}sensor-A normal temperature = 24.35°C (ts=1761201604.5129309)

Producer -> sent: {'type': 'TemperatureEvent', 'topic': 'sensor_temperature_alerts', 'sensor_id': 'sensor-A', 'temperature': 16.59, 'timestamp': 1761201605.025147}sensor-A normal temperature = 16.59°C (ts=1761201605.025147)

Producer -> sent: {'type': 'TemperatureEvent', 'topic': 'sensor_temperature_alerts', 'sensor_id': 'sensor-A', 'temperature': 28.28, 'timestamp': 1761201605.5321863}sensor-A normal temperature = 28.28°C (ts=1761201605.5321863)

Producer -> sent: {'type': 'TemperatureEvent', 'topic': 'sensor_temperature_alerts', 'sensor_id': 'sensor-C', 'temperature': 25.57, 'timestamp': 1761201606.037555}sensor-C normal temperature = 25.57°C (ts=1761201606.037555)

Producer -> sent: {'type': 'TemperatureEvent', 'topic

In [2]:
from google.colab import files
files.download('consumer_sim.log')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>